#  TP2 - Beto

## Imports

In [1]:
!pip install transformers

In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=e42ecb6a0b6ce4caf754a42352d54ddf5663a7a233a9c9ab0fb103dd6fa21f0f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
import torch
from transformers import  BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import torch.optim
import numpy as np
import pandas as pd
import time
import datetime
import random
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!wget https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/train.csv
!wget https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/test.csv

--2023-12-07 18:34:24--  https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72229641 (69M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  68.88M  84.4MB/s    in 0.8s    

2023-12-07 18:34:25 (84.4 MB/s) - ‘train.csv’ saved [72229641/72229641]

--2023-12-07 18:34:25--  https://raw.githubusercontent.com/dlevi150/7506R-2C2023-GRUPO33/main/TP2/datasets/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1

## Preprocesamiento

In [5]:

# Load the dataset into a pandas dataframe.
df = pd.read_csv('train.csv')
df = df[:10000]

In [6]:
df

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo
...,...,...,...
9995,9995,"Diversión, entretenida película sobre WWII Ale...",positivo
9996,9996,Dáme un respiro. ¿Cómo puede alguien decir que...,negativo
9997,9997,Esta película es una mala película. Pero despu...,negativo
9998,9998,Esta es una película que probablemente fue hec...,negativo


In [7]:
df['sentimiento_encodeado'] = df['sentimiento'].map({'positivo': 1, 'negativo': 0})

reviews = df['review_es']
sentiment = df['sentimiento_encodeado']

In [8]:
df.loc[df['sentimiento'] == "negativo"]

,ID,review_es,sentimiento,sentimiento_encodeado
3,3,"Básicamente, hay una familia donde un niño peq...",negativo,0
7,7,"Este espectáculo fue una idea increíble, fresc...",negativo,0
8,8,Alentados por los comentarios positivos sobre ...,negativo,0
10,10,Phil El alienígena es una de esas películas pe...,negativo,0
11,11,Vi esta película cuando tenía alrededor de 12 ...,negativo,0
...,...,...,...,...
9992,9992,"Robert Forster, normalmente un actor de caráct...",negativo,0
9994,9994,"En primer lugar, esta es la peor película que ...",negativo,0
9996,9996,Dáme un respiro. ¿Cómo puede alguien decir que...,negativo,0
9997,9997,Esta película es una mala película. Pero despu...,negativo,0


## Tokenización

In [ ]:
# Select cpu or cuda
run_on = 'cpu'
device = torch.device(run_on)

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(reviews,
sentiment, stratify=sentiment, test_size=0.2, random_state=42)

# Report datasets lenghts
print('Training set length : {}'.format(len(X_train)))
print('Validation set length : {}'.format(len(X_val)))


In [9]:
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", do_lower_case=True)

def preprocessing(dataset):
    input_ids = []
    attention_mask = []
    for doc in dataset:
        encoded_doc = tokenizer.encode_plus(doc,
                   add_special_tokens=True, max_length=115,
                   truncation=True,pad_to_max_length=True)
        input_ids.append(encoded_doc['input_ids'])
        attention_mask.append(encoded_doc['attention_mask'])
    return (torch.tensor(input_ids),
           torch.tensor(attention_mask))

# Apply preprocessing to dataset
X_train_inputs, X_train_masks = preprocessing(X_train)
X_val_inputs, X_val_masks = preprocessing(X_val)

# Report max n° tokens in a sentence
max_len = max([torch.sum(sen) for sen in X_train_masks])
print('Max n°tokens in a sentence: {0}'.format(max_len))

Training set length : 8000
Validation set length : 2000


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Max n°tokens in a sentence: 115


## Data loaders

In [25]:
batch_size = 32

y_train_labels = torch.tensor(y_train.values)
y_val_labels = torch.tensor(y_val.values)


def dataloader(x_inputs, x_masks, y_labels):
    data = TensorDataset(x_inputs, x_masks, y_labels)
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler,
                 batch_size=batch_size,
                 num_workers=0)
    return dataloader

train_dataloader = dataloader(X_train_inputs, X_train_masks,
                   y_train_labels)
val_dataloader = dataloader(X_val_inputs, X_val_masks,
                 y_val_labels)

## Modelo

In [ ]:
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed_all(value)
set_seed(42)

model = BertForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-uncased", num_labels=3, output_attentions=False, output_hidden_states=False
)

optimizer = AdamW(model.parameters(),
                  lr = 4e-5,
                  eps = 1e-6
                  )

if run_on == 'cuda':
    model.cuda()

# Define number of epochs
epochs = 3

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
            num_warmup_steps = 0,
            num_training_steps = total_steps)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#fuction to format time
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

#function to compute accuracy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## Entrenamiento

In [ ]:
def training(n_epochs, training_dataloader,
             validation_dataloader):
    # ========================================
    #               Training
    # ========================================
    print('======= Training =======')
    for epoch_i in range(0,n_epochs):
        # Perform one full pass over the training set
        print("")
        print('======= Epoch {:} / {:} ======='.format(
             epoch_i + 1, epochs))
        # Measure how long the training epoch takes.
        t0 = time.time()
        # Reset the total loss for this epoch.
        total_loss = 0

        model.train()

        print(len(training_dataloader))
        n=0
        for step, batch in enumerate(training_dataloader):
            batch_loss = 0
            # Unpack this training batch from dataloader
            #   [0]: input ids, [1]: attention masks,
            #   [2]: labels
            b_input_ids,b_input_mask, b_labels = tuple(
                                t.to(device) for t in batch)

            # Clear any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)

            # pull loss value out of the output tuple
            loss = outputs[0]
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            torch.nn.utils.clip_grad_norm_(model.parameters(),
                                            1.0)

            # Update parameters
            # ¿take a step using the computed gradient
            optimizer.step()
            scheduler.step()

            print('batch loss: {0} | avg loss: {1}'.format(
                  batch_loss, total_loss/(step+1)))
            print(n)
            n+=1

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)


        print("")
        print("  Average training loss: {0:.2f}".
             format(avg_train_loss))
        print("  Training epoch took: {:}".format(
              format_time(time.time() - t0)))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch,
        # measure accuracy on the validation set.

        print("")
        print("======= Validation =======")

        t0 = time.time()

        # Put the model in evaluation mode
        model.eval()

        # Tracking variables
        eval_loss, eval_accuracy = 0, 0
        all_logits = []
        all_labels = []
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Add batch to device
            # Unpack this training batch from our dataloader.
            #   [0]: input ids, [1]: attention masks,
            #   [2]: labels
            b_input_ids, b_input_mask, b_labels = tuple(
                                t.to(device) for t in batch)


            # Model will not to compute gradients
            with torch.no_grad():
                # Forward pass
                # This will return the logits
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask)

            # The "logits" are the output values
            # prior to applying an activation function
            logits = outputs[0]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            b_labels = b_labels.to('cpu').numpy()

            # Save batch logits and labels
            # We will use thoses in the confusion matrix
            predict_labels = np.argmax(
                             logits, axis=1).flatten()
            all_logits.extend(predict_labels.tolist())
            all_labels.extend(b_labels.tolist())

            # Calculate the accuracy for this batch
            tmp_eval_accuracy = flat_accuracy(
                                logits, b_labels)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy

        # Report the final accuracy for this validation run.
        print("  Accuracy: {0:.2f}".
              format(eval_accuracy / (step+1)))
        print("  Validation took: {:}".format(
             format_time(time.time() - t0)))

    #print the confusion matrix"
    conf = confusion_matrix(
           all_labels, all_logits, normalize='true')
    print(conf)
    print("")
    print("Training complete")

#call function to train the model
training(epochs, train_dataloader, val_dataloader)

======= Training =======

======= Epoch 1 / 3 =======
250
batch loss: 1.1391152143478394 | avg loss: 1.1391152143478394
0
batch loss: 0.9728163480758667 | avg loss: 1.055965781211853
1
batch loss: 0.8492670655250549 | avg loss: 0.9870662093162537
2
batch loss: 0.7838683724403381 | avg loss: 0.9362667500972748
3
batch loss: 0.7339709401130676 | avg loss: 0.8958075881004334
4
batch loss: 0.7274179458618164 | avg loss: 0.8677426477273306
5
batch loss: 0.7297664284706116 | avg loss: 0.848031759262085
6
batch loss: 0.6610333323478699 | avg loss: 0.8246569558978081
7
batch loss: 0.6960812211036682 | avg loss: 0.8103707631429037
8
batch loss: 0.678974449634552 | avg loss: 0.7972311317920685
9
batch loss: 0.6790268421173096 | avg loss: 0.7864852872761813
10
batch loss: 0.640371561050415 | avg loss: 0.7743091434240341
11
batch loss: 0.6610097289085388 | avg loss: 0.7655938038459191
12
batch loss: 0.6311843991279602 | avg loss: 0.7559931320803506
13
batch loss: 0.6285736560821533 | avg loss: 0.7

In [ ]:
torch.save(model, '/content/drive/MyDrive/BETOotromodelopreentrenado.pt')

In [10]:
model = torch.load('/content/drive/MyDrive/BETOotromodelopreentrenado.pt')

## Metricas

In [15]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix


In [32]:
y_pred = predict_texts(model, tokenizer, X_val, threshold=0.5)

# Calcular métricas
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

# Imprimir o utilizar estas métricas según sea necesario
print(f'Precisión: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'Exactitud: {accuracy:.4f}')

0
0
1
1
2
0
3
1
4
0
5
0
6
1
7
1
8
1
9
1
10
0
11
0
12
0
13
1
14
1
15
0
16
1
17
1
18
1
19
1
20
1
21
1
22
0
23
0
24
0
25
0
26
1
27
1
28
0
29
1
30
0
31
0
32
1
33
0
34
1
35
0
36
0
37
1
38
0
39
0
40
1
41
1
42
1
43
0
44
0
45
0
46
1
47
1
48
1
49
1
50
0
51
1
52
1
53
0
54
0
55
0
56
1
57
1
58
0
59
0
60
0
61
1
62
1
63
1
64
0
65
1
66
0
67
0
68
0
69
0
70
1
71
0
72
0
73
1
74
0
75
1
76
1
77
0
78
0
79
1
80
1
81
1
82
1
83
0
84
1
85
1
86
1
87
1
88
1
89
1
90
1
91
0
92
0
93
1
94
0
95
0
96
1
97
1
98
1
99
0
100
1
101
1
102
0
103
1
104
1
105
1
106
1
107
1
108
1
109
0
110
1
111
1
112
1
113
0
114
0
115
1
116
0
117
0
118
0
119
0
120
1
121
0
122
1
123
1
124
1
125
0
126
1
127
0
128
1
129
0
130
0
131
1
132
0
133
1
134
0
135
1
136
1
137
1
138
1
139
0
140
0
141
0
142
0
143
0
144
0
145
1
146
1
147
0
148
1
149
0
150
0
151
1
152
1
153
0
154
1
155
1
156
1
157
1
158
1
159
1
160
0
161
0
162
1
163
0
164
0
165
1
166
0
167
0
168
0
169
0
170
1
171
1
172
1
173
0
174
1
175
0
176
0
177
1
178
1
179
1
180
1
181
0
182
0
183
1
184
1


## Predicción Test

In [ ]:
df_test = pd.read_csv("test.csv")

In [ ]:
test_ids = df_test['ID']


In [30]:
def predict_texts(model, tokenizer, texts, threshold=0.5):
    # List to store predictions
    predictions = []
    n = 0
    # Put the model in evaluation mode
    model.eval()

    # Loop over each text in the list
    for text in texts:
        print(n)
        # Tokenize the text and convert to tensors
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

        # Move the tensors to the device (GPU if available)
        for key in inputs:
            inputs[key] = inputs[key].to(device)

        # Perform the forward pass
        with torch.no_grad():
            outputs = model(**inputs)

        # Get the predicted probability for class 1
        prob_class_1 = torch.sigmoid(outputs.logits[:, 1]).item()

        # Predict 1 if the probability is above the threshold, else 0
        predicted_label = 1 if prob_class_1 > threshold else 0

        # Append the prediction to the list
        predictions.append(predicted_label)
        print(predicted_label)
        n += 1

    return predictions

# Example usage
text_list = ["Texto 1 para predicción.", "Texto 2 para predicción.", ...]
predictions = predict_texts(model, tokenizer, df_test.review_es, threshold=0.5)


In [ ]:
data = {'ID': test_ids, 'sentimiento': predictions}
df_final = pd.DataFrame(data)
df_final.head(3)

,ID,sentimiento
0,60000,0
1,60001,0
2,60002,0


In [ ]:
df_final['sentimiento'] = df_final['sentimiento'].map({1: 'positivo', 0: 'negativo'})
df_final.head(3)

,ID,sentimiento
0,60000,negativo
1,60001,negativo
2,60002,negativo


In [ ]:
df_final.loc[df['sentimiento'] == "negativo"]

,ID,sentimiento
3,60003,negativo
7,60007,negativo
8,60008,negativo
10,60010,negativo
11,60011,negativo
...,...,...
8594,68594,positivo
8595,68595,positivo
8596,68596,negativo
8597,68597,positivo


In [ ]:
df_final.to_csv('submission.csv', index=False)

In [ ]:
df_final.to_csv('/content/drive/My Drive/submission-beto1.csv', index=False)

In [ ]:
df_final

,ID,sentimiento
0,60000,negativo
1,60001,negativo
2,60002,negativo
3,60003,negativo
4,60004,positivo
...,...,...
8594,68594,positivo
8595,68595,positivo
8596,68596,negativo
8597,68597,positivo
